In [25]:
import os
import moviepy.editor as mp
from pydub import AudioSegment
import numpy as np
import librosa
import soundfile as sf

In [2]:
# 提取音频
def extract_audio_from_mp4(video_path):
    video = mp.VideoFileClip(video_path)
    audio = video.audio
    audio_path = video_path.rsplit('.', 1)[0] + '.wav'
    audio.write_audiofile(audio_path)
    return audio_path

In [4]:
audio_path = extract_audio_from_mp4('C:/Users/Liudong/Desktop/MyData/4875c316bf9677f69a794ddf546e219c.mp4')

MoviePy - Writing audio in C:/Users/Liudong/Desktop/MyData/4875c316bf9677f69a794ddf546e219c.wav


MoviePy - Done.


In [18]:
# 音频合并
def merge_audio_with_length_of_b(audio_a_path, audio_b_path):
    audio_a = AudioSegment.from_file(audio_a_path)
    audio_b = AudioSegment.from_file(audio_b_path)
    # 创建 2 秒的空白音频
    silence = AudioSegment.silent(duration=2000)

    # 在音频 b 前后添加空白音频
    padded_audio_b = silence + audio_b + silence
    adjusted_audio_a = audio_a - 20

    # 取音频 b 的长度
    target_length = len(padded_audio_b)

    # 如果音频 a 长度大于音频 b，裁剪音频 a 到音频 b 的长度
    if len(adjusted_audio_a) > target_length:
        adjusted_audio_a = adjusted_audio_a[:target_length]

    adjusted_audio_b = padded_audio_b + 5

    mixed_audio = adjusted_audio_a.overlay(adjusted_audio_b)

    return mixed_audio

In [19]:
audio_a_path = "C:/Users/Liudong/Desktop/MyData/噪音视频.wav"
audio_b_path = "C:/Users/Liudong/Desktop/MyData/4875c316bf9677f69a794ddf546e219c.wav"
audio_path = audio_a_path.rsplit('.', 1)[0] + '.wav'
mixed_audio = merge_audio_with_length_of_b(audio_a_path, audio_b_path)
mixed_audio.export("C:/Users/Liudong/Desktop/MyData/mixed_audio_with_length_of_b.wav", format="wav")

<_io.BufferedRandom name='C:/Users/Liudong/Desktop/MyData/mixed_audio_with_length_of_b.wav'>

In [29]:
# 谱减法降噪
'''
假设音频开始的一部分为噪声，用于估计噪声功率谱，然后进行谱减法降噪
'''
def spectral_subtraction_without_noise_file(audio_signal, window_size=2048, hop_length=512, alpha=2):
    stft = librosa.stft(audio_signal, n_fft=window_size, hop_length=hop_length)
    magnitude = np.abs(stft)
    phase = np.angle(stft)

    # 估计噪声功率谱（假设音频开始部分为噪声）
    noise_magnitude = magnitude[:, :window_size // hop_length]
    noise_mean_magnitude = np.mean(noise_magnitude, axis=-1, keepdims=True)

    # 谱减法降噪
    subtracted_magnitude = np.maximum(magnitude - alpha * noise_mean_magnitude, 0)

    # 重构降噪后的音频信号
    subtracted_stft = subtracted_magnitude * np.exp(1j * phase)
    denoised_signal = librosa.istft(subtracted_stft, hop_length=hop_length)

    return denoised_signal

In [30]:
# 加载音频文件（假设音频文件为 wav 格式）
audio_path = "C:/Users/Liudong/Desktop/MyData/mixed_audio_with_length_of_b.wav"
audio_signal, sample_rate = librosa.load(audio_path, sr=None)

# 进行谱减法降噪
denoised_audio = spectral_subtraction_without_noise_file(audio_signal)

# 保存降噪后的音频
output_path = "C:/Users/Liudong/Desktop/MyData/denoised_audio_without_noise_file.wav"
sf.write(output_path, denoised_audio, samplerate=sample_rate)

In [31]:
# 谱减法降噪
'''
具备噪声音频
'''
def spectral_subtraction(audio_signal, noise_signal, alpha=2):
    # 计算音频信号和噪声信号的短时傅里叶变换
    audio_stft = librosa.stft(audio_signal)
    noise_stft = librosa.stft(noise_signal)

    # 计算噪声功率谱
    noise_power = np.abs(noise_stft)**2

    # 计算音频功率谱
    audio_power = np.abs(audio_stft)**2

    # 估计噪声平均功率谱
    noise_mean_power = np.mean(noise_power, axis=-1, keepdims=True)

    # 谱减法降噪
    subtracted_power = np.maximum(audio_power - alpha * noise_mean_power, 0)

    # 计算降噪后的幅度谱
    subtracted_magnitude = np.sqrt(subtracted_power)

    # 重构降噪后的音频信号
    subtracted_stft = subtracted_magnitude * np.exp(1j * np.angle(audio_stft))
    denoised_signal = librosa.istft(subtracted_stft)

    return denoised_signal

In [32]:
# 加载音频文件
audio_path = "C:/Users/Liudong/Desktop/MyData/mixed_audio_with_length_of_b.wav"
noise_path = "C:/Users/Liudong/Desktop/MyData/噪音视频.wav"
audio_signal, _ = librosa.load(audio_path, sr=None)
noise_signal, _ = librosa.load(noise_path, sr=None)

# 进行谱减法降噪
denoised_audio = spectral_subtraction(audio_signal, noise_signal)
output_path = "C:/Users/Liudong/Desktop/MyData/denoised_audio.wav"
sf.write(output_path, denoised_audio, samplerate=sample_rate)

In [33]:
# 维纳滤波
def wiener_filtering(audio_signal, noise_signal):
    # 计算音频信号和噪声信号的短时傅里叶变换
    audio_stft = librosa.stft(audio_signal)
    noise_stft = librosa.stft(noise_signal)

    # 计算噪声功率谱
    noise_power = np.abs(noise_stft)**2

    # 计算音频功率谱
    audio_power = np.abs(audio_stft)**2

    # 估计噪声平均功率谱
    noise_mean_power = np.mean(noise_power, axis=-1, keepdims=True)

    # 计算维纳滤波器系数
    filter_coeff = audio_power / (audio_power + noise_mean_power)

    # 应用维纳滤波器
    filtered_stft = filter_coeff * audio_stft

    # 重构降噪后的音频信号
    denoised_signal = librosa.istft(filtered_stft)

    return denoised_signal

In [36]:
# 加载音频文件（假设音频文件为 wav 格式）
audio_path = "C:/Users/Liudong/Desktop/MyData/mixed_audio_with_length_of_b.wav"
noise_path = "C:/Users/Liudong/Desktop/MyData/噪音视频.wav"
audio_signal, _ = librosa.load(audio_path, sr=None)
noise_signal, _ = librosa.load(noise_path, sr=None)

# 进行维纳滤波降噪
denoised_audio = wiener_filtering(audio_signal, noise_signal)

# 保存降噪后的音频
output_path = "C:/Users/Liudong/Desktop/MyData/denoised_audio_wiener_filter.wav"
sf.write(output_path, denoised_audio, samplerate=sample_rate)